In [5]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder

# Load Data

In [6]:
train = pd.read_csv('C:/Users/dideo/yeardreamsc/study_repo/5.kaggle/0002 house_price/train.csv')
test = pd.read_csv('C:/Users/dideo/yeardreamsc/study_repo/5.kaggle/0002 house_price/test.csv')
submission = pd.read_csv('C:/Users/dideo/yeardreamsc/study_repo/5.kaggle/0002 house_price/sample_submission.csv')

In [7]:
#데이터 가공 전후의 비교를 위해 copy모듈을 이용하여 원본 데이터를 보존합니다.
import copy

raw_train = copy.deepcopy(train)
raw_test = copy.deepcopy(test)

## 피쳐 엔지니어링

In [8]:
# 전처리를 위해 임시로 concat
all_data = pd.concat((train_df,test_df),axis=0).reset_index(drop=True)

In [9]:
## 새로운 파생변수의 추가 ## 

all_data['TotalSF']=all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']
all_data['No2ndFlr']=(all_data['2ndFlrSF']==0)
all_data['NoBsmt']=(all_data['TotalBsmtSF']==0)

all_data.drop(['1stFlrSF','2ndFlrSF'], axis=1, inplace=True)

In [10]:
all_data['YrBltAndRemod']=all_data['YearBuilt']+all_data['YearRemodAdd']
all_data.drop(['YrBltAndRemod','YearBuilt', 'YearRemodAdd'],axis=1)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,TotalSF,No2ndFlr,NoBsmt
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,NaN,0,2,2008,WD,Normal,208500.0,2566.0,False,False
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,NaN,0,5,2007,WD,Normal,181500.0,2524.0,True,False
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,NaN,0,9,2008,WD,Normal,223500.0,2706.0,False,False
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,NaN,0,2,2006,WD,Abnorml,140000.0,2473.0,False,False
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,NaN,0,12,2008,WD,Normal,250000.0,3343.0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,NaN,0,6,2006,WD,Normal,NaN,1638.0,False,False
2915,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,NaN,0,4,2006,WD,Abnorml,NaN,1638.0,False,False
2916,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,NaN,0,9,2006,WD,Abnorml,NaN,2448.0,True,False
2917,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,Shed,700,7,2006,WD,Normal,NaN,1882.0,True,False


In [11]:
#빌딩 넘버는 수치적인 의미라기보단 범주형 의미

all_data['MSSubClass'] = all_data['MSSubClass'].astype(str)  
all_data['MoSold'] = all_data['MoSold'].astype(str)
all_data['YrSold'] = all_data['YrSold'].astype(str)

In [12]:
# 순서의 범주를 가지는 형태를 직접 값을 할당

Basement = ['BsmtCond', 'BsmtExposure', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtFinType1', 'BsmtFinType2', 'BsmtQual', 'BsmtUnfSF', 'TotalBsmtSF']
Bsmt = all_data[Basement]

Bsmt=Bsmt.replace(to_replace='Po', value=1)
Bsmt=Bsmt.replace(to_replace='Fa', value=2)
Bsmt=Bsmt.replace(to_replace='TA', value=3)
Bsmt=Bsmt.replace(to_replace='Gd', value=4)
Bsmt=Bsmt.replace(to_replace='Ex', value=5)
Bsmt=Bsmt.replace(to_replace='None', value=0)

Bsmt = Bsmt.replace(to_replace='No', value=1)
Bsmt = Bsmt.replace(to_replace='Mn', value=2)
Bsmt = Bsmt.replace(to_replace='Av', value=3)
Bsmt = Bsmt.replace(to_replace='Gd', value=4)

Bsmt = Bsmt.replace(to_replace='Unf', value=1)
Bsmt = Bsmt.replace(to_replace='LwQ', value=2)
Bsmt = Bsmt.replace(to_replace='Rec', value=3)
Bsmt = Bsmt.replace(to_replace='BLQ', value=4)
Bsmt = Bsmt.replace(to_replace='ALQ', value=5)
Bsmt = Bsmt.replace(to_replace='GLQ', value=6)

In [13]:
## Score 파생 변수 ##

# 점수부여, 곱
Bsmt['BsmtScore'] = Bsmt['BsmtQual'] * Bsmt['BsmtCond'] * Bsmt['TotalBsmtSF']
all_data['BsmtScore'] = Bsmt['BsmtScore']

Bsmt['BsmtFin'] = (Bsmt['BsmtFinSF1'] * Bsmt['BsmtFinType1']) + (Bsmt['BsmtFinSF2'] * Bsmt['BsmtFinType2'])
all_data['BsmtFinScore'] = Bsmt['BsmtFin']
all_data['BsmtDNF']=(all_data['BsmtFinScore']==0)

all_data.drop(Basement, axis=1, inplace=True)



In [14]:
lot=['LotFrontage', 'LotArea','LotConfig','LotShape']
Lot=all_data[lot]

Lot['LotScore'] = np.log((Lot['LotFrontage'] * Lot['LotArea'])+1)
all_data['LotScore']=Lot['LotScore']

all_data.drop(lot, axis=1, inplace=True)

In [15]:
garage=['GarageArea','GarageCars','GarageCond','GarageFinish','GarageQual','GarageType','GarageYrBlt']
Garage=all_data[garage]
all_data['NoGarage']=(all_data['GarageArea']==0)


Garage=Garage.replace(to_replace='Po', value=1)
Garage=Garage.replace(to_replace='Fa', value=2)
Garage=Garage.replace(to_replace='TA', value=3)
Garage=Garage.replace(to_replace='Gd', value=4)
Garage=Garage.replace(to_replace='Ex', value=5)
Garage=Garage.replace(to_replace='None', value=0)

Garage=Garage.replace(to_replace='Unf', value=1)
Garage=Garage.replace(to_replace='RFn', value=2)
Garage=Garage.replace(to_replace='Fin', value=3)

Garage=Garage.replace(to_replace='CarPort', value=1)
Garage=Garage.replace(to_replace='Basment', value=4)
Garage=Garage.replace(to_replace='Detchd', value=2)
Garage=Garage.replace(to_replace='2Types', value=3)
Garage=Garage.replace(to_replace='Basement', value=5)
Garage=Garage.replace(to_replace='Attchd', value=6)
Garage=Garage.replace(to_replace='BuiltIn', value=7)

In [16]:
Garage['GarageScore']=(Garage['GarageArea']) * (Garage['GarageCars']) * (Garage['GarageFinish'])*(Garage['GarageQual']) * \
(Garage['GarageType'])
all_data['GarageScore']=Garage['GarageScore']


all_data.drop(garage, axis=1, inplace=True)

In [17]:
### 선형회귀를 위한 log transformation ###
# target log transformation
all_data['SalePrice'] = np.log1p(all_data['SalePrice'])

### 결측치

In [30]:
missing_value = train.isnull().sum().sort_values(ascending=False) 
missing_value = missing_value[missing_value !=0]

pd.DataFrame({'percentage': missing_value / len(train) * 100 ,  'value' : missing_value})

,percentage,value
PoolQC,99.520548,1453
MiscFeature,96.301370,1406
Alley,93.767123,1369
Fence,80.753425,1179
MasVnrType,59.726027,872
FireplaceQu,47.260274,690
LotFrontage,17.739726,259
GarageYrBlt,5.547945,81
GarageCond,5.547945,81
GarageType,5.547945,81


In [32]:
all_data.drop(['Id', 'PoolQC', 'MiscFeature', 'Alley', 'Fence'], axis=1)  # 결측치가 과도한 칼럼은 삭제

,MSSubClass,MSZoning,Street,LandContour,Utilities,LandSlope,Neighborhood,Condition1,Condition2,BldgType,...,TotalSF,No2ndFlr,NoBsmt,YrBltAndRemod,BsmtScore,BsmtFinScore,BsmtDNF,LotScore,NoGarage,GarageScore
0,60,RL,Pave,Lvl,AllPub,Gtl,CollgCr,Norm,Norm,1Fam,...,2566.0,False,False,4006,10272.0,4236.0,False,13.216311,False,39456.0
1,20,RL,Pave,Lvl,AllPub,Gtl,Veenker,Feedr,Norm,1Fam,...,2524.0,True,False,3952,15144.0,4890.0,False,13.551546,False,33120.0
2,60,RL,Pave,Lvl,AllPub,Gtl,CollgCr,Norm,Norm,1Fam,...,2706.0,False,False,4003,11040.0,2916.0,False,13.547632,False,43776.0
3,70,RL,Pave,Lvl,AllPub,Gtl,Crawfor,Norm,Norm,1Fam,...,2473.0,False,False,3885,9072.0,1080.0,False,13.258643,False,11556.0
4,60,RL,Pave,Lvl,AllPub,Gtl,NoRidge,Norm,Norm,1Fam,...,3343.0,False,False,4000,13740.0,3930.0,False,13.996031,False,90288.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,160,RM,Pave,Lvl,AllPub,Gtl,MeadowV,Norm,Norm,Twnhs,...,1638.0,False,False,3940,4914.0,0.0,True,10.612926,True,NaN
2915,160,RM,Pave,Lvl,AllPub,Gtl,MeadowV,Norm,Norm,TwnhsE,...,1638.0,False,False,3940,4914.0,756.0,False,10.590994,False,858.0
2916,20,RL,Pave,Lvl,AllPub,Gtl,Mitchel,Norm,Norm,1Fam,...,2448.0,True,False,3956,11016.0,6120.0,False,14.978662,False,6912.0
2917,85,RL,Pave,Lvl,AllPub,Gtl,Mitchel,Norm,Norm,1Fam,...,1882.0,True,False,3984,10944.0,2022.0,False,13.380632,True,NaN
